In [7]:
# Importación de librerías necesarias
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import logging
import warnings
import sys
# Configurar logging y warnings
logging.basicConfig(level=logging.WARNING, format='%(asctime)s - %(levelname)s - %(message)s')
warnings.filterwarnings('ignore')
set_log_level("ERROR")

In [8]:
# Lee el archivo como un DataFrame
data_full  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')
productos_descripcion = pd.read_csv('data/tb_productos_descripcion.txt', delimiter='\t')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1600)

# Muestra las primeras filas del DataFrame
print(data_full.head())
data_full.info()
print(df_pid_validos.head())
print(productos_descripcion.head())
productos_descripcion.info()

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_

In [9]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

# Ver cuántos registros tiene la lista
num_registros = len(product_ids_validos)
print(f"La lista product_ids_validos tiene {num_registros} registros.")

La lista product_ids_validos tiene 780 registros.


In [10]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
filtered_data = data_full[data_full['product_id'].isin(product_ids_validos)]

# Ver cuántos registros tiene el DataFrame filtrado
num_registros_filtrados = len(filtered_data)
print(f"El DataFrame filtrado tiene {num_registros_filtrados} registros.")

# Verificar que todos los product_id en el DataFrame filtrado están en la lista de productos válidos
productos_unicos_filtrados = filtered_data['product_id'].unique()
productos_invalidos = [pid for pid in productos_unicos_filtrados if pid not in product_ids_validos]

if len(productos_invalidos) == 0:
    print("Todos los product_id en el DataFrame filtrado son válidos.")
else:
    print(f"Se encontraron productos no válidos en el DataFrame filtrado: {productos_invalidos}")

# (Opcional) Ver cuántos registros hay por cada product_id
registros_por_producto = filtered_data['product_id'].value_counts()
print("Número de registros por product_id en el DataFrame filtrado:")
print(registros_por_producto)

# Ver como esta el data frame
filtered_data.info()
filtered_data

El DataFrame filtrado tiene 2293481 registros.
Todos los product_id en el DataFrame filtrado son válidos.
Número de registros por product_id en el DataFrame filtrado:
product_id
20111    7973
20122    7950
20120    7537
20326    7397
20132    7199
         ... 
21267      67
21252      67
21276      64
20886      63
20953      62
Name: count, Length: 780, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 140.0 MB


,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452
...,...,...,...,...,...,...,...
2945813,201912,10105,20853,0,1,0.02230,0.02230
2945814,201912,10092,20853,0,1,0.00669,0.00669
2945815,201912,10006,20853,0,7,0.02898,0.02898
2945816,201912,10018,20853,0,4,0.01561,0.01561


In [11]:
# 3. Fusión de los dataframes
df_merged = pd.merge(filtered_data, productos_descripcion, on='product_id', how='left')

# 4. Feature Engineering

# 4.1 Características temporales
df_merged['periodo'] = pd.to_datetime(df_merged['periodo'].astype(str), format='%Y%m')
df_merged['año'] = df_merged['periodo'].dt.year
df_merged['mes'] = df_merged['periodo'].dt.month
df_merged['trimestre'] = df_merged['periodo'].dt.quarter

# 4.2 Codificación de variables categóricas
le = LabelEncoder()
df_merged['cat1_encoded'] = le.fit_transform(df_merged['cat1'])
df_merged['cat2_encoded'] = le.fit_transform(df_merged['cat2'])
df_merged['cat3_encoded'] = le.fit_transform(df_merged['cat3'])
df_merged['brand_encoded'] = le.fit_transform(df_merged['brand'])

# 4.3 Características basadas en agrupaciones
df_merged['avg_tn_por_producto'] = df_merged.groupby('product_id')['tn'].transform('mean')
df_merged['avg_qty_por_cliente'] = df_merged.groupby('customer_id')['cust_request_qty'].transform('mean')

# 4.4 Ratios y proporciones
df_merged['ratio_tn_qty'] = df_merged['tn'] / df_merged['cust_request_qty']

# 4.5 Precios cuidados (ya es 1 o 0)
df_merged['precios_cuidados'] = df_merged['plan_precios_cuidados']

# 4.6 Categorización del tamaño del SKU por product_id
def categorize_sku_size(group):
    sku_sizes = group['sku_size'].dropna()
    if sku_sizes.empty:
        group['sku_size_category'] = 'Desconocido'
    else:
        min_size = sku_sizes.min()
        max_size = sku_sizes.max()
        
        if min_size == max_size:
            group['sku_size_category'] = 'Mediano'
        else:
            group['sku_size_category'] = pd.cut(sku_sizes, 
                                                bins=[0, min_size + (max_size - min_size)/3, 
                                                      min_size + 2*(max_size - min_size)/3, float('inf')],
                                                labels=['Pequeño', 'Mediano', 'Grande'],
                                                include_lowest=True)
    return group

df_merged = df_merged.groupby('product_id').apply(categorize_sku_size)
df_merged['sku_size_category'] = df_merged['sku_size_category'].astype('category')

# 4.7 Características de tamaño por marca
df_merged['avg_size_por_brand'] = df_merged.groupby('brand')['sku_size'].transform('mean')
df_merged['max_size_por_brand'] = df_merged.groupby('brand')['sku_size'].transform('max')
df_merged['min_size_por_brand'] = df_merged.groupby('brand')['sku_size'].transform('min')
df_merged['size_range_por_brand'] = df_merged['max_size_por_brand'] - df_merged['min_size_por_brand']

# 5. Selección de variables para NeuralProphet
regresores = [
    'año', 'mes', 'trimestre', 'cat1_encoded', 'cat2_encoded', 'cat3_encoded',
    'brand_encoded', 'avg_tn_por_producto', 'avg_qty_por_cliente', 'ratio_tn_qty',
    'precios_cuidados', 'sku_size',
    'avg_size_por_brand', 'max_size_por_brand', 'min_size_por_brand', 'size_range_por_brand'
]

# 6. Preparación del dataframe final para NeuralProphet
df_final = df_merged[['periodo', 'product_id', 'customer_id', 'tn'] + regresores].rename(columns={'periodo': 'ds', 'tn': 'y'})

# 7. Guardado del dataframe final
df_final.to_csv('df_final_para_neuralprophet.csv', index=False)

print("Dataframe final guardado con éxito. Listo para ser utilizado en NeuralProphet.")

# 8. Resumen de las nuevas características
print("\nResumen de las nuevas características:")
for column in regresores:
    if df_final[column].dtype == 'object' or df_final[column].dtype.name == 'category':
        print(f"\n{column}:")
        print(df_final[column].value_counts())
    else:
        print(f"\n{column}:")
        print(df_final[column].describe())

# 9. Verificación de valores nulos
print("\nVerificación de valores nulos:")
print(df_final.isnull().sum())

# 10. Correlaciones entre las variables
print("\nCorrelaciones entre las variables:")
correlation_matrix = df_final[['y'] + regresores].corr()
print(correlation_matrix['y'].sort_values(ascending=False))
#
# Ver el nuevo DF
df_final

Dataframe final guardado con éxito. Listo para ser utilizado en NeuralProphet.

Resumen de las nuevas características:

año:
count    2.293481e+06
mean     2.018020e+03
std      8.190902e-01
min      2.017000e+03
25%      2.017000e+03
50%      2.018000e+03
75%      2.019000e+03
max      2.019000e+03
Name: año, dtype: float64

mes:
count    2.293481e+06
mean     6.488693e+00
std      3.382992e+00
min      1.000000e+00
25%      4.000000e+00
50%      6.000000e+00
75%      9.000000e+00
max      1.200000e+01
Name: mes, dtype: float64

trimestre:
count    2.293481e+06
mean     2.489110e+00
std      1.110215e+00
min      1.000000e+00
25%      2.000000e+00
50%      2.000000e+00
75%      3.000000e+00
max      4.000000e+00
Name: trimestre, dtype: float64

cat1_encoded:
count    2.293481e+06
mean     1.366851e+00
std      7.882376e-01
min      0.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      2.000000e+00
max      3.000000e+00
Name: cat1_encoded, dtype: float64

cat2_encoded:
coun

ds  product_id  customer_id         y   año  mes  trimestre  cat1_encoded  cat2_encoded  cat3_encoded  brand_encoded  avg_tn_por_producto  avg_qty_por_cliente  ratio_tn_qty  precios_cuidados  sku_size  avg_size_por_brand  max_size_por_brand  min_size_por_brand  size_range_por_brand
product_id                                                                                                                                                                                                                                                                                                           
20001      31064   2017-01-01       20001        10001  99.43861  2017    1          1             1            10            47              0             8.156253             5.870028      9.039874                 0      3000         1834.663381                3000                 400                  2600
           31065   2017-01-01       20001        10063   0.12312  2017    1          1             1            10            47              0             8.156253             1.086374      0.123120                 0      3000         1834.663381                3000                 400                  2600
           31066   2017-01-01       20001        10080   0.24625  2017    1          1             1            10            47              0             8.156253             1.396885      0.246250                 0      3000         1834.663381                3000                 400                  2600
           31067   2017-01-01       20001        10094   1.23123  2017    1          1             1            10            47              0             8.156253             1.087856      1.231230                 0      3000         1834.663381                3000                 400                  2600
           31068   2017-01-01       20001        10184   0.06716  2017    1          1             1            10            47              0             8.156253             1.102106      0.067160                 0      3000         1834.663381                3000                 400                  2600
...                       ...         ...          ...       ...   ...  ...        ...           ...           ...           ...            ...                  ...                  ...           ...               ...       ...                 ...                 ...                 ...                   ...
21276      2230901 2019-11-01       21276        10154   0.00148  2019   11          4             2             6            18             23             0.007101             1.842218      0.001480                 0       140          201.614133                 750                  10                   740
           2285763 2019-12-01       21276        10219   0.00075  2019   12          4             2             6            18             23             0.007101             2.527533      0.000750                 0       140          201.614133                 750                  10                   740
           2285764 2019-12-01       21276        10052   0.00594  2019   12          4             2             6            18             23             0.007101             2.214567      0.005940                 0       140          201.614133                 750                  10                   740
           2285765 2019-12-01       21276        10029   0.00075  2019   12          4             2             6            18             23             0.007101            10.168633      0.000750                 0       140          201.614133                 750                  10                   740
           2285766 2019-12-01       21276        10289   0.00148  2019   12          4             2             6            18             23             0.007101             1.304869      0.001480                 0       140          201.614133                 750                  10                   7

In [6]:
# Fin del notebook de creación del nuevo DF con FE